<a href="https://colab.research.google.com/github/bwbayu/TalentMatch/blob/development/model/training/doc2vec_all_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wordsegment
!pip install num2words
!pip install deep-translator
!pip install gensim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from wordsegment import load, segment
from num2words import num2words
from deep_translator import GoogleTranslator
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# DATASET RESUME 30k NONER


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Collab Dataset/final/resume30k_noner.csv')
df.head()

,Unnamed: 0,category,clean_data,text_length
0,0,accountant,result oriented organized bilingual accounting...,550
1,1,accountant,flexible accountant adapts seamlessly constant...,865
2,2,accountant,highly analytical detail oriented professional...,699
3,3,accountant,analysis prepare auction sale journal finalize...,308
4,4,accountant,experience accounting profession bachelor degr...,482


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Step 1: Extract the text data
documents = df['clean_data'].tolist()

# Step 2: Prepare the data for training the Doc2Vec model
tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(documents)]

In [ ]:
# Step 3: Train the Doc2Vec model
%%time
modelr30knoner = Doc2Vec(tagged_data, vector_size=20, window=2, min_count=1, workers=4, epochs=50)

CPU times: user 54min 8s, sys: 39.4 s, total: 54min 48s
Wall time: 34min 57s


In [ ]:
modelr30knoner.save('/content/drive/MyDrive/Collab Dataset/final/modelr30knoner.model')

# DATASET CV HUGGING


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Collab Dataset/final/cv_hugging.csv')
df.head()

,clean_data,text_length
0,accountant professional summary result oriente...,568
1,staff accountant summary flexible accountant a...,875
2,staff accountant summary highly analytical det...,733
3,senior accountant summary highly competent mot...,512
4,senior accountant summary eleven year experien...,500


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Step 1: Extract the text data
documents = df['clean_data'].tolist()

# Step 2: Prepare the data for training the Doc2Vec model
tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(documents)]

In [ ]:
# Step 3: Train the Doc2Vec model
%%time
modelcvhugging = Doc2Vec(tagged_data, vector_size=20, window=2, min_count=1, workers=4, epochs=50)

CPU times: user 57min 17s, sys: 42.3 s, total: 58min
Wall time: 36min 45s


In [ ]:
modelcvhugging.save('/content/drive/MyDrive/Collab Dataset/final/modelcvhugging.model')

# DATASET CV combined hugging all





In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Collab Dataset/final/combined_all+hugging_all.csv')
df.head()

,clean_data,text_length
0,accountant professional summary result oriente...,568
1,staff accountant summary flexible accountant a...,875
2,staff accountant summary highly analytical det...,733
3,senior accountant summary highly competent mot...,512
4,senior accountant summary eleven year experien...,500


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Step 1: Extract the text data
documents = df['clean_data'].tolist()

# Step 2: Prepare the data for training the Doc2Vec model
tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(documents)]

In [ ]:
# Step 3: Train the Doc2Vec model
%%time
modelcvhuggingcombineall = Doc2Vec(tagged_data, vector_size=20, window=2, min_count=1, workers=4, epochs=50)

CPU times: user 1h 15min 50s, sys: 1min 15s, total: 1h 17min 5s
Wall time: 49min 38s


In [ ]:
modelcvhuggingcombineall.save('/content/drive/MyDrive/Collab Dataset/final/modelcvhuggingcombineall.model')

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from wordsegment import load, segment
from num2words import num2words
from deep_translator import GoogleTranslator
from nltk.tag import pos_tag
import spacy

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

load()
translator = GoogleTranslator(source='id', target='en')

nlp = spacy.load("en_core_web_sm")

def split_text(text, max_length):
    # Fungsi untuk membagi teks menjadi bagian yang lebih kecil
    words = text.split()
    parts = []
    current_part = []

    for word in words:
        if len(' '.join(current_part + [word])) <= max_length:
            current_part.append(word)
        else:
            parts.append(' '.join(current_part))
            current_part = [word]

    if current_part:
        parts.append(' '.join(current_part))

    return parts


def remove_verbs(text):
    doc = nlp(text)
    non_verbs = [token.text for token in doc if token.pos_ != "VERB"]
    return ' '.join(non_verbs)

def preprocessing_data(text):
  # terjemah kata

  text = text.lower()
  text = remove_verbs(text)

  text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
  text = re.sub(r'[^\w\s]', ' ', text)
  text = text.replace('s1', 'bachelor')
  text = text.replace('s2', 'master')
  text = text.replace('s3', 'doctorate')
  text = text.replace('d3', 'associate degree')
  text = text.replace('d4', 'professional degree')

  # ubah angka jadi kata
  pattern = r'\b\d+\b'

  def replace_with_words(match):
      number = int(match.group())
      return num2words(number)

  text = re.sub(pattern, replace_with_words, text)

  # pisah kata yang gabung
  # ada error
  segmented_text = segment(text)
  text = ' '.join(segmented_text)

  text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  text = text.replace('\n', ' ')
  text = text.replace('etc', ' ')

  stop_words = set(stopwords.words('english'))
  tokens = word_tokenize(text)
  tokens = [word for word in tokens if word not in stop_words]

  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(word) for word in tokens]

  preprocessed_text = ' '.join(tokens)

  # preprocessed_text = remove_verbs(preprocessed_text)

  return preprocessed_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
data = [
    ["CVGPT1 Data Science", "I am an analytical and detail-oriented Data Scientist with a strong background in data analysis, visualization, and database management. I possess proficiency in tools such as SQL, Excel, Python, and R, with a keen ability to extract meaningful insights from large datasets. I am an excellent communicator capable of conveying complex data insights to non-technical audiences. As a fresh graduate with outstanding academic performance and hands-on experience in data projects, I am eager to contribute my skills and knowledge to a dynamic team. I hold a Bachelor of Science in Data Science from a reputable university, where I graduated in June 2024 with a GPA of 3.95/4.00. My relevant coursework includes Data Analysis, Data Visualization, SQL, Python Programming, R Programming, and Database Management."],
    ["CVGPT4 FullStack Developer", "Experienced Lead Developer and Fullstack Developer with over five years of professional experience in software development and two years in leadership roles. Proficient in Java, Spring Boot, and React.js, with a strong understanding of RESTful API design and implementation. Adept at managing and mentoring development teams, overseeing the design and development of scalable web applications, and implementing CI/CD pipelines. Excellent leadership, problem-solving, and communication skills, with a proven ability to motivate and inspire team members. Bachelor of Science in Computer Science Reputable University Graduated: June 2018 Relevant Coursework: Software Development, Database Management, Agile Methodologies, Cloud Computing"],
    ["CVRIL1 Data Science","areas interest deep learning control system design programming python electric machinery web development analytics technical activities hindustan aeronautics limited bangalore weeks guidance mr satish senior engineer hangar mirage fighter aircraft technical skills programming languages matlab python java web frameworks django flask simulation software ltspice intermediate mipower intermediate version control git gitbash github data analysis notebook tools jupyter notebook database management xampp mysql basics python software packages anaconda python two python three pycharm java ide eclipse operating systems windows ubuntu debian kali linux education details january two thousand nineteen btech electrical electronics engineering manipal institute technology january two thousand fifteen deeksha center january two thousand thirteen little flower public school august two thousand manipal academy higher education experience company themathcompany description currently working casino based operator macau responsibilities include segmenting customers based value bring company providing data backed insights improved segmentation target marketing strategy skill details data analysis less than one year excel less than one year machine learning less than one year mathematics less than one year python less than one year matlab less than one year electrical engineering less than one year sql less than one year"],
    ["CVRIL2 Python Developer","technical proficiencies platform ubuntu fedora cent os windows database mysql languages python tensorflow numpy c cplusplus education details january two thousand sixteen me computer engineering pune maharashtra savitribai phule pune university january two thousand fourteen be computer engineering pune maharashtra savitribai phule pune university january two thousand ten ryk science college maharashtra state board january two thousand eight maharashtra state board python developer python developer skill details cplusplus experience six months mysql experience six months python experience six months company details company fresher description python programming"],
    ["CVRIL3 Sales","education details bachelors bachelors commerce india guru nanak high school sales manager skill details data entry experience less than one year months cold calling experience less than one year months sales experience less than one year months salesforce experience less than one year months ms office experience less than one year months company details company emperor honda description company honda cars india ltd description worked asm maruti dealership ten years currently working manager sales honda car dealership last five years good sportsmen represent college various cricket tournaments lead nagpur university cricket team also searching job car dealership cricket academy"],
    ["JDRIL1 IT Project Manager","qualifications minimum one year work experience project manager hold certification related project management professional pmp experience managing agile waterfall methodology projects banking area familiar using scrum method familiar able work collaboration tools monitor report achievement knowledge sdlc methodology knowledge iot technology good analytical thinking problem solving skill attention detail capability project management skill strong communication collaboration skills especially across customer countries good command english spoken written willing travel customer site job description lead manage various company strategic projects banking project implement execute pmbok project management body knowledge includes cost benefits calculations identify manage project risk others perform discipline monitoring achieve challenging targets quality deliveries per milestone monitor bast invoicing achievable per task milestone project manage multiple simultaneous projects indonesia per scope schedule budget followed high level customer satisfaction responsible success project delivery point escalation team customer issues pertaining success project partner customer internal technical teams resolve issue provide reporting management stakeholders project update progress financial perform risk assessment provide feedback potential issues site visit discussion stakeholders BAYU WICAKSONO qualifications minimum one year work experience project manager hold certification related project management professional pmp experience managing agile waterfall methodology projects banking area familiar using scrum method familiar able work collaboration tools monitor report achievement knowledge sdlc methodology knowledge iot technology good analytical thinking problem solving skill attention detail capability project management skill strong communication collaboration skills especially across customer countries good command english spoken written willing travel customer site job description lead manage various company strategic projects banking project implement execute pmbok project management body knowledge includes cost benefits calculations identify manage project risk others perform discipline monitoring achieve challenging targets quality deliveries per milestone monitor bast invoicing achievable per task milestone project manage multiple simultaneous projects indonesia per scope schedule budget followed high level customer satisfaction responsible success project delivery point escalation team customer issues pertaining success project partner customer internal technical teams resolve issue provide reporting management stakeholders project update progress financial perform risk assessment provide feedback potential issues site visit discussion stakeholders"],
    ["JDRIL2 Solution Architect - Data/AI","we seeking experienced solution architect lead design implementation innovative data ai solutions role collaborate stakeholders understand business technical requirements architect optimal data ai solutions responsibilities gather business technical requirements stakeholders design end end data ai solutions develop technical architecture including data stack architecture analytics environment ai determine technical feasibility validate designs against requirements select appropriate technologies considering cost scalability ease integration coordinate data scientist data engineer ai engineer etc architect data pipelines analytics architecture machine learning models apply statistical predictive modeling techniques extract insights communicate complex architecture trade offs executives stakeholders stay date technologies trends mining natural resources requirements two years experience solutions architect similar role five years experience data scientist preferable two more data platform project two more ai adoption project gen ai preferable strong statistical modeling machine learning data science skills knowledge data infrastructure pipelines storage visualization ability develop detailed technical requirements business needs excellent communication strategic thinking abilities comfortable explaining complex architectures trade offs bs ms computer science statistics analytics related field"],
    ["JDRIL3 Data Science Intern","responsibility collect process analyze large datasets extract meaningful insights trends generate monitoring report needed ensure seamless monitoring reporting cycle work cross functional teams understand data needs provide relevant insights perform data validation ensure data accuracy integrity collaborate it teams ensure availability reliability data creating database schemas represent support business process stay date industry trends best practices data analysis visualization requirement bachelors degree data science reputable university fresh graduate outstanding performance welcome proficiency data analysis tools software sql excel python r similar experience data visualization tools tableau power bi similar strong analytical problem solving skills excellent attention detail accuracy good communication interpersonal skills ability convey complex data insights non technical audiences"],
    ["JDRIL4 Sales","Develop and implement marketing plans for each channel which include crafting of content, designing advertisements, and identifying target audiences Monitor competitors' offerings to resolve how they might affect business performance Review competitor's pricing tactics to ensure that we are competitive in the market Work with vendors in making sure products are delivered timely and meet quality standards Orchestrate new programs to drive sales across multiple channels Develop strong and sustainable relationships with key accounts or other distribution channels Run day-to-day operations of a channel, including handling inventory levels, liaising with vendors, and providing customer service to customers Identify new opportunities within a channel that could increase sales through existing customers or help attract new customers to the company's products or services Managing relationships with customers to ensure satisfaction with products and services offered by the company How will you get here? 5+ years of proven experience in business development, distributor partner management, or other customer facing commercial roles Deep understanding of the Laboratory Product market in Indonesia Proficient in English; both verbal and written to communicate with English-speaking business associates Strong communicator, influencing and effective presentation skills Able to collaborate in a matrixed environment working with team with varied strengths Able to travel when needed"],
    ["JDRIL5 Data Engineer","about responsibilities role perform data exploration data cleaning data imputation feature engineering unstructured structured data build infrastructure optimal extraction transformation loading etl data wide variety data sources develop maintain optimal data pipeline architecture training statistical machine learning models regression classification develop maintain evaluations measure effectiveness training data includes measuring capabilities models variety tasks domains collaborate data scientists machine learning engineers develop comprehensive data science machine learning solution pipeline requirements need minimum qualifications bachelors degree computer science related fields equivalent software engineering experience proficiency python programming language experience dataset processing feature engineering using tools numpy pandas scikit learn visualization skills using tools matplotlib seaborn bokeh understanding deep learning frameworks pytorch tensorflow understanding sql nosql understands hadoop spark kafka hive presto proficiency source control ie git preferred make stand crowd preferred qualifications deep understanding object oriented programming oop concepts inheritance delegation abstract class understanding cloud native technologies aws gcp azure experience using docker experience using aws services s three ec two glue sagemaker experience aws step function aws lambda better proficiency scala java programming languages enjoy iterating quickly research prototypes learning new technologies"],
    ["JDRIL6 FullStack Developer","lead developer fullstack developer using java springboot react job description lead manage team developers providing technical guidance mentorship oversee design development implementation scalable web applications manage prioritize development pipeline monitor evaluate progress implement maintain ci cd pipelines using tools jenkins similar devops tools facilitate agile ceremonies sprint planning daily stand ups retrospectives identify address technical challenges bottlenecks within team resolve incidents ensure system availability within sla conduct code reviews provide constructive feedback team members foster collaborative innovative team environment stay updated emerging technologies industry trends drive continuous improvement education bachelor higher degree computer science related fields experience needed long duration needed experience length service five years professional experience software development least two years leadership role additional expertises need proficiency fullstack development expertise technologies spring boot react js strong understanding restful api design implementation experience databases oracle oracle pl sql familiarity version control systems git ci cd pipelines knowledge agile methodologies practices excellent leadership team management abilities strong problem solving skills attention detail effective communication interpersonal skills ability motivate inspire team members experience cloud services aws azure google cloud knowledge docker container orchestration familiarity project management tools jira confluence"],
]

In [ ]:
for i in data:
  i[1] = preprocessing_data(i[1])

In [ ]:
model1 = Doc2Vec.load("/content/drive/MyDrive/Collab Dataset/final/modelr30knoner.model")

In [ ]:
# Separate CVs and Job Descriptions
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))
cvs = [item for item in data if item[0].startswith("CV")]
jobs = [item for item in data if item[0].startswith("JD")]

# Infer vectors for each CV and job description
cv_vectors = [(cv[0], model1.infer_vector(cv[1].split())) for cv in cvs]
job_vectors = [(job[0], model1.infer_vector(job[1].split())) for job in jobs]

# Compare each CV with each job description and print the similarity
for cv_title, cv_vector in cv_vectors:
    for job_title, job_vector in job_vectors:
        similarity = cosine_similarity([cv_vector], [job_vector])[0][0]
        print(f"{cv_title} === {job_title}: {similarity:.4f}")

<IPython.core.display.Javascript object>

CVGPT1 Data Science === JDRIL1 IT Project Manager: 0.4057
CVGPT1 Data Science === JDRIL2 Solution Architect - Data/AI: 0.6739
CVGPT1 Data Science === JDRIL3 Data Science Intern: 0.7815
CVGPT1 Data Science === JDRIL4 Sales: 0.5866
CVGPT1 Data Science === JDRIL5 Data Engineer: 0.6753
CVGPT1 Data Science === JDRIL6 FullStack Developer: 0.3958
CVGPT4 FullStack Developer === JDRIL1 IT Project Manager: 0.5102
CVGPT4 FullStack Developer === JDRIL2 Solution Architect - Data/AI: 0.5345
CVGPT4 FullStack Developer === JDRIL3 Data Science Intern: 0.4320
CVGPT4 FullStack Developer === JDRIL4 Sales: 0.5158
CVGPT4 FullStack Developer === JDRIL5 Data Engineer: 0.4891
CVGPT4 FullStack Developer === JDRIL6 FullStack Developer: 0.7426
CVRIL1 Data Science === JDRIL1 IT Project Manager: 0.3298
CVRIL1 Data Science === JDRIL2 Solution Architect - Data/AI: 0.5980
CVRIL1 Data Science === JDRIL3 Data Science Intern: 0.7626
CVRIL1 Data Science === JDRIL4 Sales: 0.5190
CVRIL1 Data Science === JDRIL5 Data Engineer

In [ ]:
model2 = Doc2Vec.load("/content/drive/MyDrive/Collab Dataset/final/modelcvhugging.model")

In [ ]:
# Separate CVs and Job Descriptions
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))
cvs = [item for item in data if item[0].startswith("CV")]
jobs = [item for item in data if item[0].startswith("JD")]

# Infer vectors for each CV and job description
cv_vectors = [(cv[0], model1.infer_vector(cv[1].split())) for cv in cvs]
job_vectors = [(job[0], model1.infer_vector(job[1].split())) for job in jobs]

# Compare each CV with each job description and print the similarity
for cv_title, cv_vector in cv_vectors:
    for job_title, job_vector in job_vectors:
        similarity = cosine_similarity([cv_vector], [job_vector])[0][0]
        print(f"{cv_title} === {job_title}: {similarity:.4f}")

<IPython.core.display.Javascript object>

CVGPT1 Data Science === JDGPT1 Data Science Intern: 0.7948
CVGPT1 Data Science === JDGPT2 FullStack Developer: 0.4378
CVGPT1 Data Science === JDRIL1 IT Project Manager: 0.4396
CVGPT1 Data Science === JDRIL2 Solution Architect - Data/AI: 0.7118
CVGPT1 Data Science === JDRIL3 Data Science Intern: 0.8121
CVGPT1 Data Science === JDRIL4 Sales: 0.6145
CVGPT1 Data Science === JDRIL5 Data Engineer: 0.6891
CVGPT1 Data Science === JDRIL6 FullStack Developer: 0.4497
CVGPT2 customer service === JDGPT1 Data Science Intern: 0.5876
CVGPT2 customer service === JDGPT2 FullStack Developer: 0.4712
CVGPT2 customer service === JDRIL1 IT Project Manager: 0.6013
CVGPT2 customer service === JDRIL2 Solution Architect - Data/AI: 0.4309
CVGPT2 customer service === JDRIL3 Data Science Intern: 0.6030
CVGPT2 customer service === JDRIL4 Sales: 0.7468
CVGPT2 customer service === JDRIL5 Data Engineer: 0.3375
CVGPT2 customer service === JDRIL6 FullStack Developer: 0.4931
CVGPT3 Data Science and customer service === JDG

In [ ]:
model3 = Doc2Vec.load("/content/drive/MyDrive/Collab Dataset/final/modelcvhuggingcombineall.model")

In [ ]:
# Separate CVs and Job Descriptions
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))
cvs = [item for item in data if item[0].startswith("CV")]
jobs = [item for item in data if item[0].startswith("JD")]

# Infer vectors for each CV and job description
cv_vectors = [(cv[0], model1.infer_vector(cv[1].split())) for cv in cvs]
job_vectors = [(job[0], model1.infer_vector(job[1].split())) for job in jobs]

# Compare each CV with each job description and print the similarity
for cv_title, cv_vector in cv_vectors:
    for job_title, job_vector in job_vectors:
        similarity = cosine_similarity([cv_vector], [job_vector])[0][0]
        print(f"{cv_title} === {job_title}: {similarity:.4f}")

<IPython.core.display.Javascript object>

CVGPT1 Data Science === JDGPT1 Data Science Intern: 0.7754
CVGPT1 Data Science === JDGPT2 FullStack Developer: 0.4125
CVGPT1 Data Science === JDRIL1 IT Project Manager: 0.4177
CVGPT1 Data Science === JDRIL2 Solution Architect - Data/AI: 0.6822
CVGPT1 Data Science === JDRIL3 Data Science Intern: 0.7674
CVGPT1 Data Science === JDRIL4 Sales: 0.5749
CVGPT1 Data Science === JDRIL5 Data Engineer: 0.6609
CVGPT1 Data Science === JDRIL6 FullStack Developer: 0.4138
CVGPT2 customer service === JDGPT1 Data Science Intern: 0.6230
CVGPT2 customer service === JDGPT2 FullStack Developer: 0.4880
CVGPT2 customer service === JDRIL1 IT Project Manager: 0.6054
CVGPT2 customer service === JDRIL2 Solution Architect - Data/AI: 0.4401
CVGPT2 customer service === JDRIL3 Data Science Intern: 0.6074
CVGPT2 customer service === JDRIL4 Sales: 0.7612
CVGPT2 customer service === JDRIL5 Data Engineer: 0.3427
CVGPT2 customer service === JDRIL6 FullStack Developer: 0.4863
CVGPT3 Data Science and customer service === JDG